## 데이터셋 API 호출 함수

In [1]:
import requests

def DATA_browse(API_KEY, svc_id):
    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"+svc_id
    params = {"key": API_KEY}

    # print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    data = res.json()
    print(data)
    return data

def DATA_search(API_KEY, query: str):

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
    params = {"key": API_KEY, "query": "korean text", "from": 0, "size": 5}
    # key / CHAR / 필수 / API_KEY
    # query / CHAR / 필수 / 검색키워드
    # from / CHAR / 옵션 / 페이지시작위치
    # size / CHAR / 옵션 / 페이지사이즈

    # print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    data = res.json()
    print(data)
    return data

## 논문 토큰 호출용 클래스 정의

In [2]:
import json
import base64
from Crypto.Cipher import AES
from urllib.parse import quote
import requests

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str

## 토큰 호출 함수

In [3]:
from datetime import datetime
# 나중에 AESTestClass도 호출해야 함.

def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")
    print(mac)

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')
    print(dt)

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    print("🔎 호출 URL 미리보기:", requests.Request('GET', endpoint, params=params).prepare().url)
    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()
    print("HTTP", response.status_code)

    # 결과 출력
    # print(response.text)

    data = response.json()
    print(data)
    token = data['access_token'] 
    return token

## test: access_token 호출

In [4]:
# import os
# from dotenv import load_dotenv

# # 환경변수 호출
# load_dotenv(override=True)
# CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
# API_KEY = os.getenv("SCIENCEON_API_KEY")
# MAC_ADDRESS = os.getenv("MAC_ADDRESS")

# access_token = call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID)
# access_token

## API 호출 함수(토큰 상태 확인하고 써야 함.)

In [5]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)

# 관련 논문 검색 함수
def ARTI_search(client_id, token, query):
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "search",
        "target": "ARTI",
        "searchQuery": query,
        # -----
        # "sortField": "",
        # "pubyear" : 발행일(내림차순)
        # "title" : 논문명(오름차순)
        # "jtitle" : 저널명(오름차순)
        # "" : 정확도(내림차순)
        'curPage': 1, # 현재페이지 번호
        'rowCount': 10, # 디스플레이 건수(기본값 10, 최대값 100)
        # 'session_id': "", # 이건 뭔지 모르겠
        # 'include': 'Publisher,JournalName,ISSN',
        # 'exclude': None, # include와 동시에 사용 불가
        # 'grouping': 'Y' # 안 쓸 거면 비우기
    }
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    xml = res.text
    return xml


# 단일 논문 조회 함수
def ARTI_browse(client_id, token, cn):
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": cn,
        "include": "",
        "exclude": None, # include와 동시에 사용 불가
    }
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    xml = res.text
    return xml

# XML to DF 함수
def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    # DataFrame 생성
    df = pd.DataFrame(records)
    return df

## test: ARTI_search

In [6]:
# import json

# # QUery 만드는 부분: 따로 함수를 만들어야 함.
# query = {
#     "BI":"AI", 
#     "KW":"AI"
# }
# # 쿼리 인코딩
# json_query = json.dumps(query, separators=(',', ':')) 

# search_xml = ARTI_search(CLIENT_ID, access_token, json_query)
# search_df = xml_to_df(search_xml)
# display(search_df)

## test: browse

In [7]:
# cn = "JAKO202117242229851"

# search_xml = ARTI_browse(CLIENT_ID, access_token, cn)
# search_df = xml_to_df(search_xml)
# display(search_df)

## 최종본

In [9]:
import os
import sys
from dotenv import load_dotenv

# 환경변수 호출
load_dotenv(override=True)
CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")
DATA_BROWSE_KEY = os.getenv("DATAON_META_API_KEY")
DATA_SEARCH_KEY = os.getenv("DATAON_SEARCH_API_KEY")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)
access_token
max_invalid = 4
invalid_count = 0

while True:
    choice = input("입력데이터 선택\n 1. 데이터셋  2. 논문\n숫자(1 또는 2)를 입력하세요: ").strip()

    if choice == "1":
        # 데이터셋 호출 API
        svc_id = input("데이터셋 id :") # 입력예시: b37f0c9413eeb7c45f6fe31cbe3a41ef
        print(f"데이터셋을 호출합니다...with ID: {svc_id}")
        data = DATA_browse(DATA_BROWSE_KEY, svc_id)
        display(data) # json
        break

    elif choice == "2":
        # 논문 호출 API
        cn = input("논문 id :") # 입력예시: JAKO200411922932805
        print(f"논문을 호출합니다...with ID: {cn}")
        xml = ARTI_browse(CLIENT_ID, access_token, cn)
        df = xml_to_df(xml)
        display(df) # pandas df
        break
        
    else:
        invalid_count += 1
        remaining = max_invalid - invalid_count
        if remaining <= 0:
            print("입력 오류가 너무 많아 프로그램을 종료합니다.")
            sys.exit(1)
        else:
            print(f"잘못된 입력입니다. {remaining}번 남았습니다. 다시 입력해주세요.")

1C-57-DC-50-BB-31
20251009183607
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/tokenrequest.do?accounts=a5eEQ8WEyYGpUQBklLLZeGsyVz986GpHU08zH5gIHDpLWiSlp7tnZRYstpgCWvBUCwQvDfWIteJ3u4fbAzIqlQ%3D%3D&client_id=e37c8354a854dad4e74fe1238b2921f8e47e34ddcaacbd3243d01caa93e874c8
HTTP 200
{'access_token': 'c529013bacee48736e1cc484fcacc13dbf5e780f13f17459174db7863327ad68', 'refresh_token': 'cc8f9e398500386f3443dfa54eedb3226cac700d432806fb9782d6b8d33a109a', 'refresh_token_expire': '2025-10-23 18:36:07.810', 'access_token_expire': '2025-10-09 20:36:07.810', 'issued_at': '2025-10-09 18:36:07.000810', 'client_id': 'e37c8354a854dad4e74fe1238b2921f8e47e34ddcaacbd3243d01caa93e874c8'}
논문을 호출합니다...with ID: JAKO200411922932805
🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=e37c8354a854dad4e74fe1238b2921f8e47e34ddcaacbd3243d01caa93e874c8&token=c529013bacee48736e1cc484fcacc13dbf5e780f13f17459174db7863327ad68&version=1.0&action=browse&target=ARTI&cn=JAKO200411922932805&include=
HTTP 2

,CN,DBCode,JournalId,Publisher,JournalName,ISSN,ISBN,VolumeId,VolNo1,VolNo2,...,FulltextFlag,AbstractFlag,PageInfo,DOI,FulltextURL,ContentURL,MobileURL,Keyword,Degree,None
0,JAKO200411922932805,JAKO,NJOU00290524,대한기계학회,大韓機械學會論文集. Transactions of the Korean Society ...,1226-4873;2288-5226;,,57,28,11,...,2,1,pp.1664-1671,https://doi.org/10.3795/ksme-a.2004.28.11.1664,,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,단층촬영사진 . 한국인 무릎 관절 모델 . 인공 관절 치환술,,
